My question is about Resource constraint Project Scheduling Problem code in cplex. I am trying to apply Preemption to it

https://stackoverflow.com/questions/54684892/my-question-is-about-resource-constraint-project-scheduling-problem-code-in-cple#:~:text=First%2C%20you%20should%20add%20some,t%2Ci

Constraint programming solution: CP Optimizer does not directly toggle a “preemptive” setting, but we can model preemptive tasks by breaking them into optional sub-intervals and linking them with constraints. A typical CP modeling approach for fully preemptive RCPSP is:

Segment the tasks: Represent each task as a group of smaller sub-tasks (intervals) that collectively constitute the whole task. Use a span constraint to link them – for example, span(TaskInterval, [Part1, Part2, ...]) ensures the high-level task interval spans exactly all its present sub-intervals
stackoverflow.com
. In effect, the task’s start time aligns with the first segment’s start, and its end aligns with the last segment’s end.

Enforce sequential order: Impose ordering among a task’s segments so they do not overlap. Typically, we add endBeforeStart(Part_i, Part_{i+1}) for each consecutive pair of sub-activities
stackoverflow.com
. This creates a chain, meaning the task’s parts execute one after the other (but possibly with idle gaps in between, allowing other tasks to use the resource).

Conserve total duration: Ensure the sum of all segment durations equals the original task duration (so that no work is lost or gained by preemption). This can be done by a constraint like sum(lengthOf(Part_i for all i)) == TaskDuration
stackoverflow.com
. In practice, one might allow the overall task interval to be longer than its sum of work (to account for pauses) but still require that the total “processing time” across all segments matches the task’s required effort.

Hexaly: 

https://www.hexaly.com/templates/multi-mode-resource-constrained-project-scheduling-mrcpsp
https://www.hexaly.com/templates/preemptive-resource-constrained-project-scheduling-problem-prcpsp
https://www.hexaly.com/templates/resource-constrained-project-scheduling-problem-rcpsp

Alternative schedules with alterante:
https://www.ibm.com/docs/en/cofz/12.9.0?topic=concepts-constraints-groups-interval-variables-in-cp-optimizer
house_building_optional.py

